# Giới thiệu chung về thuật toán
### CatBoost là một biến thể gradient boosting trên Decision Tree, được thiết kế để xử lý biến phân loại một cách “thông minh” và giảm overfitting / target leakage tốt hơn các thuật toán như XGBoost, LightGBM

#### Gradient boosting (GBDT) = xây nhiều cây nhỏ tuần tự:

- Cây đầu học dự đoán mục tiêu.

- Cây sau học sửa lỗi (residual / gradient) của tổ hợp các cây trước.

- Cuối cùng mô hình = tổng (hoặc tổ hợp) của tất cả cây.

#### Vấn đề của bản GBDT “truyền thống”:

- Dễ overfit.

- Xử lý biến phân loại phải one-hot / label encoding → mất thông tin hoặc gây nhiễu.

- Dễ bị target leakage / prediction shift khi dùng thống kê mục tiêu (mean target theo category) được tính từ cả tập train.

### Điểm cải tiến của CatBoost
#### Oblivious (symmetric) trees
- Ở mỗi tầng, tất cả node dùng cùng một điều kiện split.

- Cây sâu d luôn có 2^d lá; đường đi biểu diễn bằng dãy bit (trái/phải).
#### Xử lý biến phân loại bằng Target Statistics (TS)
- Với một category c, thay vì one-hot, ta dùng:

-   $TS(c)≈ \frac{∑y+a⋅Pn+a}{n + a}$

#### Ordered Boosting
- CatBoost trộn ngẫu nhiên các mẫu thành một permutation.

- Đối với mỗi mẫu tại vị trí i, TS được tính chỉ từ các mẫu đứng trước nó (1…i-1) trong permutation

- Khi xây cây thứ k, dự đoán và gradient cho một mẫu được tính từ mô hình dựa trên prefix trước đó trong permutation, thay vì toàn bộ
	​


## Bước 1 - Import các thư viện cần thiết

In [ ]:
import pandas as pd
import os
import joblib
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

## Bước 2 - Khai báo mô hình
Sử dụng model CatBoostRegressor từ thư viện catboost

In [14]:
base_model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    bootstrap_type="Bayesian",
    random_seed=42,
    verbose=False,      # tắt spam log khi CV
    od_type="Iter",
    od_wait=100         # early stopping trong từng lần train
)

## Bước 3 - Chuẩn bị dữ liệu cho mô hình



### 3.1. Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [ ]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

### 3.2. Chuẩn bị dữ liệu

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [ ]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

## Bước 4 - Xây dựng và huấn luyện mô hình Cat Boost

### 4.1. Bộ tham số dùng để tìm kiếm

In [15]:
# =========  KHOẢNG TÌM KIẾM THAM SỐ (PARAM DISTRIBUTIONS) =========
param_dist = {
    "depth": [4, 5, 6, 7, 8, 9, 10],
    "learning_rate": np.linspace(0.01, 0.15, 15),
    "l2_leaf_reg": np.logspace(0, 3, 20),      # 1 -> 1000
    "bagging_temperature": np.linspace(0, 5, 11),
    "border_count": [64, 128, 254],
    "iterations": [1000, 2000, 3000, 4000]     # kết hợp với early stopping
}


### 4.2. Khai báo RandomizedSearchCV

In [ ]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=40,     # số tổ hợp random thử (tăng nếu muốn kỹ hơn)
    scoring="neg_root_mean_squared_error",
    cv=5,          # 5-fold CV
    n_jobs=-1,
    refit=True,
    random_state=42
)

### 4.3. Fit RandomizedSearchCV với training data

In [ ]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  25.7s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  27.8s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  32.3s
[CV] END bagging_temperature=2.0, border_count=254, depth=5, iterations=4000, l2_leaf_reg=54.55594781168517, learning_rate=0.15; total time=  32.7s
[CV] END bagging_temperature=2.5, border_count=254, depth=7, iterations=2000, l2_leaf_reg=54.55594781168517, learning_rate=0.03; total time=  33.0s
[CV] END bagging_temperature=2.5, border_count=254, depth=7, iterations=2000, l2_leaf_reg=54.55594781168517, learning_rate=0.03; total time=  35.6s
[CV] END bagging_temperature=2.5, border_count=64,

### 4.4. Bộ tham số tốt nhất 

In [ ]:
print("Best params:", random_search.best_params_)
print("Best CV RMSE:", -random_search.best_score_)

best_cat = random_search.best_estimator_

NameError: name 'random_search' is not defined

### 4.5. Đánh giá mô hình tối ưu trên tập train

In [ ]:
y_pred = best_cat.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Decision Tree:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 loss: {train_r2:.3f}")

- Best params: {'learning_rate': np.float64(0.06999999999999999), 'l2_leaf_reg': np.float64(2.976351441631318), 'iterations': 2000, 'depth': 9, 'border_count': 64, 'bagging_temperature': np.float64(0.5)}
- Best CV RMSE: 1.9391243329337335


- Test MAE : 0.956
- Test RMSE: 1.386
- Test R²  : 0.973

### 4.6. Lưu mô hình Cat Boost đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models nếu chưa có
os.makedirs('../model/7_catboost', exist_ok=True)

# Lưu mô hình
model_path = '../model/catboost/catboost.pkl'
joblib.dump(best_cat, model_path)

## Kết luận

### Tổng kết:
1. Mô hình CatBoost đã được xây dựng và tối ưu hóa thành công
2. RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
3. Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2